### PART 1: PHPMyAdmin

Questions 1-9 below are completed in the PHPMyAdmin interface.

In [ ]:
# Q1: Some of the facilities charge a fee to members, but some do not.
# Write a SQL query to produce a list of the names of the facilities that do.
SELECT name AS facility, membercost
FROM Facilities
WHERE membercost != 0.0;

In [ ]:
# Q2: How many facilities do not charge a fee to members?
SELECT COUNT(name)
FROM Facilities
WHERE membercost = 0.0;

In [ ]:
# Q3: Write an SQL query to show a list of facilities that charge a fee to members,
# where the fee is less than 20% of the facility's monthly maintenance cost.
# Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.
SELECT facid, name AS facility, membercost, monthlymaintenance
FROM Facilities
WHERE membercost != 0.0 AND membercost < 0.2 * monthlymaintenance;

In [ ]:
# Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
# Try writing the query without using the OR operator.
SELECT *
FROM Facilities
WHERE facid IN (1, 5);

In [ ]:
# Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', 
# depending on if their monthly maintenance cost is more than $100. 
# Return the name and monthly maintenance of the facilities in question.
SELECT name AS Facility, 
CASE WHEN monthlymaintenance <= 100 THEN 'cheap'
     WHEN monthlymaintenance > 100 THEN 'expensive'
     END AS 'Monthly Maintenance'
FROM Facilities;

In [ ]:
# Q6: You'd like to get the first and last name of the last member(s) who signed up. 
# Try not to use the LIMIT clause for your solution.
SELECT firstname AS "First name",
       surname AS "Last name"
FROM Members
ORDER BY joindate DESC;

In [ ]:
# Q7: Produce a list of all members who have used a tennis court.
# Include in your output the name of the court, and the name of the member
# formatted as a single column. Ensure no duplicate data, and order by the member name.
SELECT DISTINCT CONCAT_WS(" ", m.firstname, m.surname) AS Member,
       f.name AS Facility
FROM Bookings AS b
	INNER JOIN Facilities AS f
		ON b.facid = f.facid
	INNER JOIN Members as m
		ON b.memid = m.memid
WHERE f.name LIKE 'Tennis%'
ORDER BY Member, Facility;

In [ ]:
# Q8: Produce a list of bookings on the day of 2012-09-14 which
# will cost the member (or guest) more than $30. Remember that guests have
# different costs to members (the listed costs are per half-hour 'slot'), and
# the guest user's ID is always 0. Include in your output the name of the
# facility, the name of the member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.
SELECT f.name AS facility,
       CONCAT_WS(" ", m.firstname, m.surname) AS Member,
       CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost END AS Cost
FROM Bookings AS b
	INNER JOIN Facilities AS f
		ON b.facid = f.facid
    INNER JOIN Members AS m
		ON b.memid = b.memid
WHERE DATE(b.starttime) = '2012-09-14' 
	AND (CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
              ELSE b.slots * f.membercost END) > 30
ORDER BY Cost DESC;

In [ ]:
# Q9: This time, produce the same result as in Q8, but using a subquery.
SELECT Facility,
       Member,
       Cost
FROM
	(SELECT f.name AS Facility,
     		CONCAT_WS(" ", m.firstname, m.surname) AS Member,
     		b.starttime AS bookdate,
       		CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
            	 ELSE b.slots * f.membercost END AS Cost
     FROM Bookings AS b
	 INNER JOIN Facilities AS f
		ON b.facid = f.facid
     Inner JOIN Members AS m
		ON b.memid = b.memid
     ) AS subq
WHERE DATE(bookdate) = '2012-09-14' AND Cost > 30
ORDER BY Cost DESC;

### PART 2: SQLite

The country club data was exported from PHPMyAdmin, and connected to a local SQLite instance from Jupyter notebook for the following questions. 

In [1]:
# Import necessary module
from sqlalchemy import create_engine
import pandas as pd

# Create engine to connect to SQLite database
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [2]:
table_names = engine.table_names()
table_names

['Bookings', 'Facilities', 'Members']

In [3]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. 
# Remember that there's a different cost for guests and members!
df = pd.read_sql_query('SELECT name AS Facility, Revenue As "Revenue < $1000" FROM (SELECT name, SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) AS Revenue FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid GROUP BY name) AS subq WHERE Revenue < 1000', engine)
df

,Facility,Revenue < $1000
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


In [4]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
df = pd.read_sql_query('SELECT m1.surname || ", " || m1.firstname AS Member, m2.surname || ", " || m2.firstname AS "Recommended By" FROM Members as m1 INNER JOIN Members as m2 ON m1.recommendedby = m2.memid WHERE m1.recommendedby != "" ORDER BY m1.surname, m1.firstname', engine)
df

,Member,Recommended By
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


In [5]:
# Q12: Find the facilities with their usage by member, but not guests
df = pd.read_sql_query('SELECT f.name AS Facility, SUM(CASE WHEN b.memid != 0 THEN b.slots END) / 2 AS "Hour used by members" FROM Bookings AS b INNER JOin Facilities AS f ON b.facid = f.facid GROUP BY f.name ORDER BY f.name', engine)
df

,Facility,Hour used by members
0,Badminton Court,543
1,Massage Room 1,442
2,Massage Room 2,27
3,Pool Table,428
4,Snooker Table,430
5,Squash Court,209
6,Table Tennis,397
7,Tennis Court 1,478
8,Tennis Court 2,441


In [6]:
# Q13: Find the facilities usage by month, but not guests
df = pd.read_sql_query('SELECT strftime("%m",starttime) AS Month, f.name AS Facility, SUM(CASE WHEN b.memid != 0 THEN b.slots END) / 2 AS "Hour used by members" FROM Bookings AS b INNER JOin Facilities AS f ON b.facid = f.facid GROUP BY Month, Facility', engine)
df

,Month,Facility,Hour used by members
0,07,Badminton Court,82
1,07,Massage Room 1,83
2,07,Massage Room 2,4
3,07,Pool Table,55
4,07,Snooker Table,70
5,07,Squash Court,25
6,07,Table Tennis,49
7,07,Tennis Court 1,100
8,07,Tennis Court 2,61
9,08,Badminton Court,207
